In [ ]:
import gmql as gl

In [ ]:
# loading of a dataset
data_path = "/home/luca/Documenti/resources/hg_narrowPeaks"
output_path = "/home/luca/Documenti/resources/result"

parser = gl.parsers.NarrowPeakParser()

data = gl.GMQLDataset(parser=parser)\
                .load_from_path(path=data_path, meta_load=False)
    
# selects all the data coming from cell K562 and having antibody H3K4me3
condition = (data.MetaField("cell") == 'K562') & (data.MetaField("antibody") == 'H3K4me3')
filtered_data = data.meta_select(predicate=condition) # test

In [ ]:
joined = filtered_data.join(experiment=filtered_data, 
                   genometric_predicate=[gl.DLE(1000), gl.DOWN()], 
                   output="RIGHT")

In [ ]:
joined = joined.materialize(output_path=output_path)